<a href="https://colab.research.google.com/github/victorsjc/data-science-labs/blob/main/labresults-analysis-data-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar as bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import re
import unicodedata
from typing import Dict, List, Tuple, Optional
import json
from googletrans import Translator
import time

In [ ]:
class LOINCMapper:
    """
    Algoritmo para mapeamento de exames clínicos e laboratoriais para códigos LOINC
    usando embeddings e técnicas híbridas de similaridade semântica.
    """

    def __init__(self, model_name: str = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
                 loinc_csv_path: Optional[str] = None):
        """
        Inicializa o mapeador LOINC.

        Args:
            model_name: Nome do modelo de embeddings para usar
            loinc_csv_path: Caminho para o arquivo CSV da base LOINC oficial
        """
        self.model = SentenceTransformer(model_name)
        self.vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=5000)
        self.translator = Translator()

        # Carrega base LOINC (oficial ou exemplo)
        if loinc_csv_path:
            self.loinc_database = self._load_official_loinc_database(loinc_csv_path)
        else:
            self.loinc_database = self._create_sample_loinc_database()

        self.loinc_embeddings = None
        self.loinc_tfidf_matrix = None

        # Gera sinônimos em português automaticamente
        self.medical_synonyms = self._generate_portuguese_synonyms()

        self._prepare_loinc_embeddings()

    def _create_sample_loinc_database(self) -> pd.DataFrame:
        """Cria uma base de dados LOINC simplificada para demonstração."""
        sample_data = [
            {
                'loinc_code': '33747-0',
                'component': 'Hemoglobina',
                'property': 'MCnc',
                'time': 'Pt',
                'system': 'Sangue',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Hemoglobina [Massa/volume] no Sangue',
                'synonyms': 'Hb, hemoglobina total, concentração de hemoglobina'
            },
            {
                'loinc_code': '789-8',
                'component': 'Eritrócitos',
                'property': 'NCnc',
                'time': 'Pt',
                'system': 'Sangue',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Eritrócitos [#/volume] no Sangue',
                'synonyms': 'RBC, hemácias, contagem de eritrócitos, glóbulos vermelhos'
            },
            {
                'loinc_code': '6690-2',
                'component': 'Leucócitos',
                'property': 'NCnc',
                'time': 'Pt',
                'system': 'Sangue',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Leucócitos [#/volume] no Sangue',
                'synonyms': 'WBC, glóbulos brancos, contagem de leucócitos'
            },
            {
                'loinc_code': '2093-3',
                'component': 'Colesterol',
                'property': 'MCnc',
                'time': 'Pt',
                'system': 'Soro',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Colesterol [Massa/volume] no Soro ou Plasma',
                'synonyms': 'colesterol total, colesterolemia'
            },
            {
                'loinc_code': '33762-9',
                'component': 'Proteína C reativa',
                'property': 'MCnc',
                'time': 'Pt',
                'system': 'Soro',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Proteína C reativa [Massa/volume] no Soro ou Plasma',
                'synonyms': 'PCR, CRP, proteína C-reativa'
            },
            {
                'loinc_code': '2160-0',
                'component': 'Creatinina',
                'property': 'MCnc',
                'time': 'Pt',
                'system': 'Soro',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Creatinina [Massa/volume] no Soro ou Plasma',
                'synonyms': 'creatinina sérica'
            },
            {
                'loinc_code': '33746-2',
                'component': 'Ácido ascórbico',
                'property': 'MCnc',
                'time': 'Pt',
                'system': 'Soro',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Ácido ascórbico [Massa/volume] no Soro ou Plasma',
                'synonyms': 'vitamina C, ascorbato, ácido L-ascórbico, vitamina C sérica'
            },
            {
                'loinc_code': '5907-5',
                'component': 'Ácido ascórbico',
                'property': 'MCnc',
                'time': 'Pt',
                'system': 'Plasma',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Ácido ascórbico [Massa/volume] no Plasma',
                'synonyms': 'vitamina C plasmática, ascorbato plasmático, concentração vitamina C'
            },
            {
                'loinc_code': '20567-4',
                'component': 'Ácido ascórbico',
                'property': 'MCnc',
                'time': 'Pt',
                'system': 'Sangue total',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Ácido ascórbico [Massa/volume] no Sangue total',
                'synonyms': 'vitamina C total, ascorbato total, vitamina C no sangue'
            },
            {
                'loinc_code': '35668-3',
                'component': 'Ácido ascórbico',
                'property': 'MCnc',
                'time': 'Pt',
                'system': 'Urina',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Ácido ascórbico [Massa/volume] na Urina',
                'synonyms': 'vitamina C urinária, ascorbato urinário, excreção vitamina C'
            },
            {
                'loinc_code': '47132-6',
                'component': 'Ácido ascórbico',
                'property': 'MRto',
                'time': '24H',
                'system': 'Urina',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Ácido ascórbico [Massa/tempo] na Urina de 24 horas',
                'synonyms': 'excreção de vitamina C 24h, clearance vitamina C, eliminação ascorbato'
            }
        ]
        return pd.DataFrame(sample_data)

    def _load_official_loinc_database(self, csv_path: str) -> pd.DataFrame:
        """
        Carrega a base oficial LOINC de um arquivo CSV.

        Args:
            csv_path: Caminho para o arquivo LOINC.csv oficial

        Returns:
            DataFrame com os dados LOINC processados
        """
        print("📁 Carregando base LOINC oficial...")

        try:
            # Lê o arquivo CSV oficial LOINC
            df = pd.read_csv(csv_path, dtype=str, low_memory=False)

            # Mapeia colunas principais do LOINC
            column_mapping = {
                'LOINC_NUM': 'loinc_code',
                'COMPONENT': 'component',
                'PROPERTY': 'property',
                'TIME_ASPCT': 'time',
                'SYSTEM': 'system',
                'SCALE_TYP': 'scale',
                'METHOD_TYP': 'method',
                'LONG_COMMON_NAME': 'display_name',
                'SHORTNAME': 'short_name'
            }

            # Renomeia colunas se existirem
            available_columns = {k: v for k, v in column_mapping.items() if k in df.columns}
            df = df.rename(columns=available_columns)

            # Filtra apenas registros com dados essenciais
            essential_columns = ['loinc_code', 'component', 'system', 'display_name']
            df = df.dropna(subset=[col for col in essential_columns if col in df.columns])

            # Limita o dataset para performance (remove este filtro em produção)
            if len(df) > 1000:
                print(f"⚠️  Base muito grande ({len(df)} registros). Usando amostra de 1000 para demonstração.")
                df = df.head(1000)

            # Adiciona coluna de sinônimos vazia (será preenchida automaticamente)
            df['synonyms'] = ''

            print(f"✅ Base LOINC carregada com {len(df)} registros")
            return df

        except Exception as e:
            print(f"❌ Erro ao carregar base LOINC: {e}")
            print("🔄 Usando base de exemplo...")
            return self._create_sample_loinc_database()

    def _generate_portuguese_synonyms(self) -> Dict[str, List[str]]:
        """
        Gera automaticamente sinônimos em português a partir da base LOINC.

        Returns:
            Dicionário com sinônimos gerados
        """
        print("🔄 Gerando sinônimos em português...")

        synonyms_dict = {}

        # Sinônimos base manuais para termos comuns
        base_synonyms = {
            'hemoglobina': ['hb', 'hemoglobina total', 'concentração hemoglobina'],
            'eritrocitos': ['rbc', 'hemacias', 'globulos vermelhos', 'contagem eritrocitos'],
            'leucocitos': ['wbc', 'globulos brancos', 'contagem leucocitos'],
            'colesterol': ['colesterol total', 'colesterolemia'],
            'proteina c reativa': ['pcr', 'crp', 'proteina c-reativa'],
            'creatinina': ['creatinina serica', 'creat'],
            'glicose': ['glicemia', 'glucose', 'açucar no sangue'],
            'triglicerideos': ['trigliceridos', 'tg'],
            'ureia': ['ureia serica', 'nitrogenio ureico']
        }

        # Adiciona sinônimos automáticos baseados na base LOINC
        if not self.loinc_database.empty:
            unique_components = self.loinc_database['component'].dropna().unique()

            for component in unique_components[:50]:  # Limita para performance
                if pd.isna(component) or len(component.strip()) < 3:
                    continue

                normalized_component = self._normalize_text(component)

                # Gera variações automáticas
                variations = self._generate_component_variations(component)

                if variations:
                    synonyms_dict[normalized_component] = variations

        # Combina sinônimos base com os gerados
        synonyms_dict.update(base_synonyms)

        print(f"✅ {len(synonyms_dict)} grupos de sinônimos gerados")
        return synonyms_dict

    def _generate_component_variations(self, component: str) -> List[str]:
        """
        Gera variações automáticas para um componente LOINC.

        Args:
            component: Nome do componente

        Returns:
            Lista de variações/sinônimos
        """
        variations = []

        try:
            # Tradução para português se necessário
            if self._is_english(component):
                translated = self._safe_translate(component)
                if translated and translated.lower() != component.lower():
                    variations.append(translated.lower())

            # Variações de formatação
            component_lower = component.lower()
            variations.extend([
                component_lower,
                component_lower.replace(' ', ''),  # Remove espaços
                component_lower.replace('-', ' '),  # Hífen por espaço
                component_lower.replace('_', ' ')   # Underscore por espaço
            ])

            # Abreviações comuns
            abbreviations = self._generate_abbreviations(component)
            variations.extend(abbreviations)

            # Remove duplicatas e valores vazios
            variations = list(set([v for v in variations if v and len(v) > 1]))

        except Exception as e:
            print(f"⚠️  Erro ao gerar variações para '{component}': {e}")

        return variations[:10]  # Limita número de variações

    def _is_english(self, text: str) -> bool:
        """Verifica se o texto parece estar em inglês."""
        english_indicators = ['blood', 'serum', 'plasma', 'urine', 'volume', 'mass', 'concentration']
        text_lower = text.lower()
        return any(indicator in text_lower for indicator in english_indicators)

    def _safe_translate(self, text: str, max_retries: int = 3) -> Optional[str]:
        """
        Traduz texto de forma segura com tratamento de erros.

        Args:
            text: Texto a ser traduzido
            max_retries: Número máximo de tentativas

        Returns:
            Texto traduzido ou None se falhar
        """
        for attempt in range(max_retries):
            try:
                time.sleep(0.1)  # Evita rate limiting
                result = self.translator.translate(text, src='en', dest='pt')
                return result.text if result else None

            except Exception as e:
                if attempt < max_retries - 1:
                    time.sleep(1)  # Espera antes de tentar novamente
                    continue
                else:
                    print(f"⚠️  Erro na tradução de '{text}': {e}")
                    return None

        return None

    def _generate_abbreviations(self, component: str) -> List[str]:
        """Gera abreviações possíveis para um componente."""
        abbreviations = []

        # Abreviações por primeiras letras
        words = component.split()
        if len(words) > 1:
            # Primeira letra de cada palavra
            abbrev = ''.join(word[0].lower() for word in words if word)
            if len(abbrev) > 1:
                abbreviations.append(abbrev)

        # Abreviações conhecidas
        known_abbrevs = {
            'hemoglobina': ['hb'],
            'proteina c reativa': ['pcr', 'crp'],
            'aspartate aminotransferase': ['ast', 'tgo'],
            'alanine aminotransferase': ['alt', 'tgp'],
            'lactate dehydrogenase': ['ldh'],
            'creatine kinase': ['ck', 'cpk'],
            'white blood cells': ['wbc'],
            'red blood cells': ['rbc']
        }

        component_lower = component.lower()
        for term, abbrevs in known_abbrevs.items():
            if term in component_lower:
                abbreviations.extend(abbrevs)

        return abbreviations

    def _normalize_text(self, text: str) -> str:
        """Normaliza texto removendo acentos e padronizando."""
        if not text:
            return ""

        # Remove acentos
        text = unicodedata.normalize('NFD', text)
        text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')

        # Converte para minúsculas e remove caracteres especiais
        text = re.sub(r'[^\w\s]', ' ', text.lower())
        text = re.sub(r'\s+', ' ', text).strip()

        return text

    def _expand_synonyms(self, text: str) -> str:
        """Expande texto com sinônimos médicos."""
        normalized_text = self._normalize_text(text)
        words = normalized_text.split()

        expanded_terms = [normalized_text]

        for key, synonyms in self.medical_synonyms.items():
            if key in normalized_text:
                for synonym in synonyms:
                    expanded_terms.append(normalized_text.replace(key, synonym))

        return ' '.join(expanded_terms)

    def _prepare_loinc_embeddings(self):
        """Prepara embeddings e matriz TF-IDF para a base LOINC."""
        if self.loinc_database.empty:
            print("⚠️  Base LOINC vazia. Não é possível preparar embeddings e TF-IDF.")
            self.loinc_embeddings = None
            self.loinc_tfidf_matrix = None
            return

        # Prepara textos expandidos para embedding
        loinc_texts = []
        for _, row in self.loinc_database.iterrows():
            combined_text = f"{row['display_name']} {row['component']}"
            if 'synonyms' in row and pd.notna(row['synonyms']):
                 combined_text += f" {row['synonyms']}"
            expanded_text = self._expand_synonyms(combined_text)
            loinc_texts.append(expanded_text)

        if not loinc_texts:
             print("⚠️  Nenhum texto gerado para embeddings. Base LOINC pode estar vazia após processamento.")
             self.loinc_embeddings = None
             self.loinc_tfidf_matrix = None
             return

        # Gera embeddings
        self.loinc_embeddings = self.model.encode(loinc_texts)

        # Gera matriz TF-IDF
        # Fit the vectorizer first with the texts
        self.vectorizer.fit(loinc_texts)
        # Then transform the texts
        self.loinc_tfidf_matrix = self.vectorizer.transform(loinc_texts)


    def _semantic_similarity(self, query_text: str) -> np.ndarray:
        """Calcula similaridade semântica usando embeddings."""
        if self.loinc_embeddings is None:
            return np.array([])
        expanded_query = self._expand_synonyms(query_text)
        query_embedding = self.model.encode([expanded_query])

        similarities = cosine_similarity(query_embedding, self.loinc_embeddings)[0]
        return similarities

    def _lexical_similarity(self, query_text: str) -> np.ndarray:
        """Calcula similaridade lexical usando TF-IDF."""
        if self.loinc_tfidf_matrix is None:
            return np.array([])
        expanded_query = self._expand_synonyms(query_text)
        query_tfidf = self.vectorizer.transform([expanded_query])

        similarities = cosine_similarity(query_tfidf, self.loinc_tfidf_matrix)[0]
        return similarities

    def map_to_loinc(self, exam_text: str, top_k: int = 3,
                     semantic_weight: float = 0.7,
                     lexical_weight: float = 0.3) -> List[Dict]:
        """
        Mapeia texto de exame para códigos LOINC.

        Args:
            exam_text: Texto do exame a ser mapeado
            top_k: Número de melhores correspondências a retornar
            semantic_weight: Peso da similaridade semântica
            lexical_weight: Peso da similaridade lexical

        Returns:
            Lista de dicionários com códigos LOINC e scores de similaridade
        """
        if not exam_text.strip() or self.loinc_database.empty or self.loinc_embeddings is None or self.loinc_tfidf_matrix is None:
            return []

        # Calcula similaridades
        semantic_scores = self._semantic_similarity(exam_text)
        lexical_scores = self._lexical_similarity(exam_text)

        # Combina scores (técnica híbrida)
        combined_scores = (semantic_weight * semantic_scores +
                          lexical_weight * lexical_scores)

        # Obtém os top_k resultados
        top_indices = np.argsort(combined_scores)[::-1][:top_k]

        results = []
        for idx in top_indices:
            loinc_row = self.loinc_database.iloc[idx]
            result = {
                'loinc_code': loinc_row['loinc_code'],
                'display_name': loinc_row['display_name'],
                'component': loinc_row['component'],
                'system': loinc_row['system'],
                'similarity_score': float(combined_scores[idx]),
                'semantic_score': float(semantic_scores[idx]),
                'lexical_score': float(lexical_scores[idx]),
                'confidence': self._calculate_confidence(combined_scores[idx])
            }
            results.append(result)

        return results

    def _calculate_confidence(self, score: float) -> str:
        """Calcula nível de confiança baseado no score."""
        if score >= 0.8:
            return 'Alta'
        elif score >= 0.6:
            return 'Média'
        elif score >= 0.4:
            return 'Baixa'
        else:
            return 'Muito Baixa'

    def batch_mapping(self, exam_texts: List[str]) -> Dict[str, List[Dict]]:
        """Mapeia múltiplos exames em lote."""
        results = {}
        for i, text in enumerate(exam_texts):
            results[f'exam_{i}'] = self.map_to_loinc(text)
        return results

    def evaluate_mapping(self, test_cases: List[Dict]) -> Dict:
        """
        Avalia performance do mapeamento com casos de teste.

        Args:
            test_cases: Lista de dicts com 'text' e 'expected_loinc'

        Returns:
            Métricas de avaliação
        """
        if self.loinc_database.empty or self.loinc_embeddings is None or self.loinc_tfidf_matrix is None:
             print("⚠️  Não é possível avaliar: Base LOINC vazia ou não processada.")
             return {'accuracy': 0.0, 'correct_predictions': 0, 'total_cases': len(test_cases)}

        correct_predictions = 0
        total_cases = len(test_cases)

        for case in test_cases:
            predictions = self.map_to_loinc(case['text'], top_k=1)
            if predictions and predictions[0]['loinc_code'] == case['expected_loinc']:
                correct_predictions += 1

        accuracy = correct_predictions / total_cases if total_cases > 0 else 0

        return {
            'accuracy': accuracy,
            'correct_predictions': correct_predictions,
            'total_cases': total_cases
        }

In [ ]:
# Exemplo de uso e demonstração
def demonstrate_loinc_mapping():
    """Demonstra o funcionamento do algoritmo com exemplos."""

    # Inicializa o mapeador
    mapper = LOINCMapper(loinc_csv_path='/content/sample_data/ptBR11LinguisticVariant.csv')

    # Exemplos de resultados de exames
    sample_exams = [
        "Hemoglobina: 12.5 g/dL",
        "Contagem de eritrócitos: 4.2 milhões/μL",
        "WBC: 7500/μL",
        "Colesterol total: 180 mg/dL",
        "PCR: 5.2 mg/L",
        "Creatinina sérica: 1.1 mg/dL",
        "Dosagem de hemoglobina no sangue",
        "Vitamina C: 31.9 mg/L"
    ]

    print("=== DEMONSTRAÇÃO DO ALGORITMO DE MAPEAMENTO LOINC ===\n")

    for i, exam in enumerate(sample_exams, 1):
        print(f"Exemplo {i}: {exam}")
        print("-" * 50)

        results = mapper.map_to_loinc(exam, top_k=2)

        if results:
            for j, result in enumerate(results, 1):
                print(f"  {j}º Melhor match:")
                print(f"    LOINC: {result['loinc_code']}")
                print(f"    Nome: {result['display_name']}")
                print(f"    Componente: {result['component']}")
                print(f"    Sistema: {result['system']}")
                print(f"    Score Total: {result['similarity_score']:.3f}")
                print(f"    Confiança: {result['confidence']}")
                print()
        else:
            print("  Nenhuma correspondência encontrada.\n")

        print("=" * 60 + "\n")

    # Demonstra avaliação com casos de teste
    print("=== AVALIAÇÃO DE PERFORMANCE ===\n")

    test_cases = [
        {'text': 'Hemoglobina no sangue', 'expected_loinc': '33747-0'},
        {'text': 'Contagem de glóbulos vermelhos', 'expected_loinc': '789-8'},
        {'text': 'Leucócitos totais', 'expected_loinc': '6690-2'}
    ]

    evaluation = mapper.evaluate_mapping(test_cases)
    print(f"Acurácia: {evaluation['accuracy']:.2%}")
    print(f"Casos corretos: {evaluation['correct_predictions']}/{evaluation['total_cases']}")


In [ ]:
demonstrate_loinc_mapping()

📁 Carregando base LOINC oficial...
✅ Base LOINC carregada com 0 registros
🔄 Gerando sinônimos em português...
✅ 9 grupos de sinônimos gerados
⚠️  Base LOINC vazia. Não é possível preparar embeddings e TF-IDF.
=== DEMONSTRAÇÃO DO ALGORITMO DE MAPEAMENTO LOINC ===

Exemplo 1: Hemoglobina: 12.5 g/dL
--------------------------------------------------
  Nenhuma correspondência encontrada.


Exemplo 2: Contagem de eritrócitos: 4.2 milhões/μL
--------------------------------------------------
  Nenhuma correspondência encontrada.


Exemplo 3: WBC: 7500/μL
--------------------------------------------------
  Nenhuma correspondência encontrada.


Exemplo 4: Colesterol total: 180 mg/dL
--------------------------------------------------
  Nenhuma correspondência encontrada.


Exemplo 5: PCR: 5.2 mg/L
--------------------------------------------------
  Nenhuma correspondência encontrada.


Exemplo 6: Creatinina sérica: 1.1 mg/dL
--------------------------------------------------
  Nenhuma corresp

# New Section